# Easy Fine-Tuning GPT-3.5 Turbo for Robust and Reliable Clinical Language Modeling. (Medical LLM) 
Nitya Singh

### Transforming the dataset

In [ ]:
import csv
import json

def csv_to_jsonl(csv_file_path, jsonl_file_path):
    with open(csv_file_path, 'r') as csv_file:
        reader = csv.DictReader(csv_file)
        data = list(reader)

    with open(jsonl_file_path, 'w') as jsonl_file:
        for row in data:
            message = {
                "messages": [
                    {"role": "user", "content": row["user"]},
                    {"role": "assistant", "content": row["assistant"]}
                ]
            }
            jsonl_file.write(json.dumps(message) + '\n')

csv_to_jsonl('data.csv', 'output.jsonl')

### Uploading the dataset

In [ ]:
import requests

# Replace these with your actual values
api_key = 'YOUR_API_KEY'
file_path = r'path_to_the_json_file'

headers = {
    'Authorization': f'Bearer {api_key}',
}

data = {
    'purpose': 'fine-tune',
}

files = {
    'file': open(file_path, 'rb'),
}

response = requests.post('https://api.openai.com/v1/files', headers=headers, data=data, files=files)

# This will print the status code
print(response.status_code)
print(response.json())

### Status of the code 
It takes few minutes for the file to be ready for the training. Once uploaded we create a fine-tuning job. from the output 'id' is important. extract the id in a format like "file-xxxx…" and paste into the next code' training_field_id attribute.

In [ ]:
import requests
import json

# Replace these with your actual values
api_key = 'YOUR_API_KEY'
training_file_id = 'file-id'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}',
}

data = {
    'training_file': training_file_id,
    'model': 'gpt-3.5-turbo-0613',
}

response = requests.post('https://api.openai.com/v1/fine_tuning/jobs', headers=headers, data=json.dumps(data))

# This will print the status code
print(response.json())

Wait for few minutes for the training to get completed. From the previous code output use the 'id' key to get the status of the fine tuning job below.

In [ ]:
import requests
headers = {
    'Authorization': 'Bearer YOUR_API_KEY',
}

responsem = requests.get('https://api.openai.com/v1/fine_tuning/jobs/'paste your id from above here', headers=headers)

print(responsem.json())

Once completed you will get a 'fined_tuned_model' name. It has the format of ft:gpt-3.5-turbo:my-org:custom_suffix:id .This would be the name of your custom model and be used for calling through the API.
### Testing :
Change the model variable.

In [ ]:
import openai

openai.api_key = 'YOU_API_KEY'

response2 = openai.ChatCompletion.create(
  model="ft:gpt-3.5-turbo:my-org:custom_suffix:id",
  messages=[
        {"role": "system", "content": "You are a health assistant."},
        {"role": "user", "content": "What causes Gastritis ?"},
    ]
)

print(response2['choices'][0])